
# Катастрофическое забывание

## Цель: Проверить влияние fine-tuning на исходную модель.

## Описание/Пошаговая инструкция выполнения домашнего задания:
1. Скачать датасет `ImageNette`: https://github.com/fastai/imagenette (`ImageNette` это подвыборка из 10 классов датасета `ImageNet`).
2. Взять предобученную на обычном `ImageNet` модель (например, `ResNet18`) и заменить число классов на 10.
3. Дообучить модель на 10 классах `ImageNette` и замерить точность (эта точность будет считаться базовой). Можно обучить как всю модель, так и только последний слой.
4. Сохранить последний слой на 10 классов (слой классификации).
Используя код с лекции дообучить модель классифицировать датасет `CIFAR10`.
5. Вернуть оригинальный последний слой модели и проверить качество на `ImageNette` и сравнить с базовой точностью.
6. Дообучить только последний слой (отключить градиент для всех слоев кроме последнего) на `ImageNette` и проверить удалось ли добиться исходного качества.
7. Сделать выводы.

### Критерии оценки:
__Принято__ - задание выполнено полностью.

__Возвращено на доработку__ - задание не выполнено полностью.


In [108]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm

## Шаг 1 Подготовка датасета

### 1.1 Определение настроек

In [112]:
TRAIN_DIR_IMAGENETTE = 'DataForModel/imagenette2-320/train'
VAL_DIR_IMAGENETTE = 'DataForModel/imagenette2-320/val'
TRAIN_DIR_CIFAR10 = 'DataForModel/cifar10/train'
VAL_DIR_CIFAR10 = 'DataForModel/cifar10/val'
BATCH_SIZE = 32
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

### 1.2 Датасет ImageNette

In [113]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset_imagenette = torchvision.datasets.ImageFolder(TRAIN_DIR_IMAGENETTE, transform=transform)
val_dataset_imagenette = torchvision.datasets.ImageFolder(VAL_DIR_IMAGENETTE, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset_imagenette, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset_imagenette, batch_size=BATCH_SIZE)

### 1.3 Датасет CIFAR10

In [116]:
train_dataset_cifar = torchvision.datasets.CIFAR10(root=TRAIN_DIR_CIFAR10, train=True, download=True, transform=transform)
test_dataset_cifar = torchvision.datasets.CIFAR10(root=VAL_DIR_CIFAR10, train=False, download=True, transform=transform)

cifar_train = torch.utils.data.DataLoader(train_dataset_cifar, BATCH_SIZE, shuffle=True), 
cifar_test = torch.utils.data.DataLoader(test_dataset_cifar, BATCH_SIZE)

100.0%
100.0%


## Шаг 2: Замена последнего слоя на 10 классов

In [ ]:
model = torchvision.models.resnet18(pretrained=True)
original_fc = model.fc  # Сохранение оригинального слоя
torch.save(original_fc.state_dict(), 'original_fc.pth')

for param in model.parameters():
    param.requires_grad = False  # Выключение градиенты для всех параметров

model.fc = nn.Linear(model.fc.in_features, 10)  # Замена последнего слоя
model.fc.requires_grad = True  # Включаем градиенты только для нового слоя
model = model.to(DEVICE)

c:\Users\KharitWinPC\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\KharitWinPC\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Шаг 3: Обучение на ImageNette (последний слой)

### Шаг 3.1 Обучение модели

In [106]:
# Расчет точности
def evaluate(model, val_loader):
    model.eval() 
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [107]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.0001)

for epoch in range(5):
    model.train()
    for images, labels in tqdm(train_loader, desc=f"Эпоха {epoch+1}"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    acc = evaluate(model, val_loader)
    print(f"Точность валидации: {acc:.2f}%")

base_accuracy = acc
torch.save(model.fc.state_dict(), 'imagenette_fc.pth')
print(f"\nБазовая точность: {base_accuracy:.2f}%\n")

Эпоха 1: 100%|██████████| 296/296 [04:16<00:00,  1.16it/s]


Точность валидации: 91.54%


Эпоха 2:   8%|▊         | 24/296 [00:21<03:58,  1.14it/s]


KeyboardInterrupt: 

## Шаг 4: Обучение на CIFAR10

In [ ]:
model_cifar = torchvision.models.resnet18(pretrained=True)
model_cifar.fc.load_state_dict(torch.load('imagenette_fc.pth'))
model_cifar = model_cifar.to(DEVICE)

# Замораживаем все слои кроме последнего
for param in model_cifar.parameters():
    param.requires_grad = False
model_cifar.fc.requires_grad = True

optimizer_cifar = torch.optim.Adam(model_cifar.fc.parameters(), lr=0.0001)  # Только последний слой

## Шаг 5: Возврат оригинального слоя и проверка качества

In [ ]:
# Загружаем оригинальный последний слой ImageNet (1000 классов)
model.fc.load_state_dict(torch.load('original_fc.pth'))

# Проверяем точность на ImageNette с оригинальным слоем
acc_original = evaluate(model, val_loader)
print(f"\nТочность с оригинальным слоем ImageNet: {acc_original:.2f}%")
print(f"Базовая точность (наш обученный слой): {base_accuracy:.2f}%")